# 1 Introducción y Set-up

## 1.1 Los pasos del preprocesamiento
1. Establacer **target** (qué variable quieres predicir)
2. **Seleccionar variables** (cúales variables quieres usar para predecir)
3. **Oversampling** (si hay desbalance de clases, se añaden más ejemplos de la clase minoritaria)
4. **Subsampling** (si hay desbalance de clases, se eliminan ejemplos de la clase mayoritaria)
5. Eliminar variables **constantes** (si hay variables que no cambian, se eliminan porque no aportan valor)
6. Eliminar variables **no informativas** (si hay variables que no aportan información, se eliminan)
7. Rellenar **missing values** (si hay valores nulos, se rellenan con la media, mediana, moda, etc)
8. **Agrupar** categorías (si hay categorías con muy pocos ejemplos, se agrupan en una categoría "otros")
9. **One Hot Encoding** (si hay variables categóricas, se convierten a variables numéricas)
10. **Escalado** de variables (si hay variables con diferentes escalas, se escalan a la misma escala)
11. Eliminar variables **irrelevantes** (si hay variables que no aportan valor según sus correlaciones, se eliminan)
12. Eliminar variables **redundantes** (si hay variables que son redundantes (alta correlación entre ellas), se eliminan)
13. **Variable Importance** (hacer un ranking de los variables)
14. **Weighting** (darle más peso a las variables más importantes)
15. **PCA** (si hay muchas variables, se reducen dimensiones/ a un número menor de variables)

## 1.2 Librerías

In [17]:
# Load Libraries
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import pandas as pd 
from fancyimpute import IterativeImputer as MICE
from sklearn.model_selection import train_test_split

from multiprocessing import cpu_count

## 1.3 Leer datos y verificar tipos de datos

In [18]:
# Leer datos
dat = pd.read_csv("datasets/dat_sanidad_raw.csv", sep = "|")
dat

,gravedad,pct_mortalidad_norma,edad_dias,numproc,potencial_ambul,proc,estancia_esperada,tipgrd,tiping,exitus
0,4,"0,408730159",12596.0,21,0,1,NaN,Q,1,0
1,4,NaN,20973.0,22,0,1,99.0,Q,1,0
2,4,"0,278481013",19611.0,19,0,1,NaN,Q,1,0
3,3,"0,150289017",13583.0,22,0,1,100.0,Q,1,0
4,1,"0,016573296",18042.0,2,0,1,NaN,Q,1,0
...,...,...,...,...,...,...,...,...,...,...
32701,2,"0,028365137",23619.0,2,0,1,NaN,M,1,0
32702,1,"0,000605816",3935.0,1,0,1,NaN,M,1,0
32703,2,"0,040451895",30163.0,4,0,1,NaN,M,1,0
32704,2,0,29012.0,4,0,1,NaN,M,1,0


In [19]:
target = 'exitus'
categorical_variables = ['gravedad', 'tipgrd', 'potencial_ambul', 'tiping']
numeric_variables = ['pct_mortalidad_norma', 'edad_dias', 'numproc', 'estancia_esperada']

In [20]:
dat.dtypes

gravedad                  int64
pct_mortalidad_norma     object
edad_dias               float64
numproc                   int64
potencial_ambul           int64
proc                      int64
estancia_esperada       float64
tipgrd                   object
tiping                    int64
exitus                    int64
dtype: object

In [21]:
# Convertir variable pct_mortalidad_norma a numérica
dat['pct_mortalidad_norma'] = dat['pct_mortalidad_norma'].str.replace(',','.').astype(float)
dat

,gravedad,pct_mortalidad_norma,edad_dias,numproc,potencial_ambul,proc,estancia_esperada,tipgrd,tiping,exitus
0,4,0.408730,12596.0,21,0,1,NaN,Q,1,0
1,4,NaN,20973.0,22,0,1,99.0,Q,1,0
2,4,0.278481,19611.0,19,0,1,NaN,Q,1,0
3,3,0.150289,13583.0,22,0,1,100.0,Q,1,0
4,1,0.016573,18042.0,2,0,1,NaN,Q,1,0
...,...,...,...,...,...,...,...,...,...,...
32701,2,0.028365,23619.0,2,0,1,NaN,M,1,0
32702,1,0.000606,3935.0,1,0,1,NaN,M,1,0
32703,2,0.040452,30163.0,4,0,1,NaN,M,1,0
32704,2,0.000000,29012.0,4,0,1,NaN,M,1,0


## 2 Detectar y tratar missing values

In [22]:
# Contar el porcentaje de los valores no informados
## apply: Aplica una función a cada columna
dat.apply(lambda x: 100*np.sum(x.isna())/len(x)) 

gravedad                 0.000000
pct_mortalidad_norma    10.111295
edad_dias                1.449275
numproc                  0.000000
potencial_ambul          0.000000
proc                     0.000000
estancia_esperada       84.323977
tipgrd                   0.000000
tiping                   0.000000
exitus                   0.000000
dtype: float64

### 2.1 Eliminar missing values de edad_dias

In [23]:
# Eliminar missing values para la variable: edad_dias
dat = dat[dat.edad_dias.notna()]
dat

,gravedad,pct_mortalidad_norma,edad_dias,numproc,potencial_ambul,proc,estancia_esperada,tipgrd,tiping,exitus
0,4,0.408730,12596.0,21,0,1,NaN,Q,1,0
1,4,NaN,20973.0,22,0,1,99.0,Q,1,0
2,4,0.278481,19611.0,19,0,1,NaN,Q,1,0
3,3,0.150289,13583.0,22,0,1,100.0,Q,1,0
4,1,0.016573,18042.0,2,0,1,NaN,Q,1,0
...,...,...,...,...,...,...,...,...,...,...
32701,2,0.028365,23619.0,2,0,1,NaN,M,1,0
32702,1,0.000606,3935.0,1,0,1,NaN,M,1,0
32703,2,0.040452,30163.0,4,0,1,NaN,M,1,0
32704,2,0.000000,29012.0,4,0,1,NaN,M,1,0


### 2.2 Reemplacer valores categoricas por moda: potencial_ambul

In [24]:
# Calcular la moda de cada variable categórica y guardar en un diccionario
modes = dat[categorical_variables].apply(lambda x: stats.mode(x)[0][0]).to_dict()
modes

C:\Users\franz\AppData\Local\Temp\ipykernel_15700\3628653551.py:2: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  modes = dat[categorical_variables].apply(lambda x: stats.mode(x)[0][0]).to_dict()
c:\Users\franz\anaconda3\lib\site-packages\scipy\stats\_stats_py.py:110: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  warnings.warn("The input array could not be properly "
C:\Users\franz\AppData\Local\Temp\ipykernel_15700\3628653551.py:2: DeprecationWarning: Support for non-numeric arrays has been deprecated as of SciPy 1.9.0 and will be removed in 1.11.0. `pandas.DataFr

{'gravedad': 1, 'tipgrd': 'M', 'potencial_ambul': 0, 'tiping': 1}

In [25]:
# Reemplazar los valores no informados por la moda
dat = dat.fillna(value = modes, axis = 0)
dat

,gravedad,pct_mortalidad_norma,edad_dias,numproc,potencial_ambul,proc,estancia_esperada,tipgrd,tiping,exitus
0,4,0.408730,12596.0,21,0,1,NaN,Q,1,0
1,4,NaN,20973.0,22,0,1,99.0,Q,1,0
2,4,0.278481,19611.0,19,0,1,NaN,Q,1,0
3,3,0.150289,13583.0,22,0,1,100.0,Q,1,0
4,1,0.016573,18042.0,2,0,1,NaN,Q,1,0
...,...,...,...,...,...,...,...,...,...,...
32701,2,0.028365,23619.0,2,0,1,NaN,M,1,0
32702,1,0.000606,3935.0,1,0,1,NaN,M,1,0
32703,2,0.040452,30163.0,4,0,1,NaN,M,1,0
32704,2,0.000000,29012.0,4,0,1,NaN,M,1,0


### 2.3 Eliminar columna 'estancia_esperada' por la falta de más de 84% de instansias  

In [26]:
dat = dat.drop('estancia_esperada', axis=1)
dat.head()  
numeric_variables = ['pct_mortalidad_norma', 'edad_dias', 'numproc']

### 2.4 Reemplazar valores con MICE: pct_mortalidad_norma

In [27]:
# Imputación por MICE
dat_new = dat.copy()
dat_new[numeric_variables] = MICE().fit_transform(dat[numeric_variables]) #fit_transform: Ajusta el modelo y devuelve los valores imputados
dat_new

,gravedad,pct_mortalidad_norma,edad_dias,numproc,potencial_ambul,proc,tipgrd,tiping,exitus
0,4,0.408730,12596.0,21.0,0,1,Q,1,0
1,4,0.178418,20973.0,22.0,0,1,Q,1,0
2,4,0.278481,19611.0,19.0,0,1,Q,1,0
3,3,0.150289,13583.0,22.0,0,1,Q,1,0
4,1,0.016573,18042.0,2.0,0,1,Q,1,0
...,...,...,...,...,...,...,...,...,...
32701,2,0.028365,23619.0,2.0,0,1,M,1,0
32702,1,0.000606,3935.0,1.0,0,1,M,1,0
32703,2,0.040452,30163.0,4.0,0,1,M,1,0
32704,2,0.000000,29012.0,4.0,0,1,M,1,0


In [28]:
# Comprobar: Contar el porcentaje de los valores no informados
## apply: Aplica una función a cada columna
dat_new.apply(lambda x: 100*np.sum(x.isna())/len(x))

gravedad                0.0
pct_mortalidad_norma    0.0
edad_dias               0.0
numproc                 0.0
potencial_ambul         0.0
proc                    0.0
tipgrd                  0.0
tiping                  0.0
exitus                  0.0
dtype: float64

In [32]:
#dat['edad_dias'] = dat['edad_dias'].astype(int)
# cambiar tipo de variables edad_dias y numproc de float a int
dat_new['edad_dias'] = pd.to_numeric(dat_new['edad_dias'],  downcast='integer')
dat_new['numproc'] = pd.to_numeric(dat_new['numproc'],  downcast='integer')
dat_new


,gravedad,pct_mortalidad_norma,edad_dias,numproc,potencial_ambul,proc,tipgrd,tiping,exitus
0,4,0.408730,12596,21,0,1,Q,1,0
1,4,0.178418,20973,22,0,1,Q,1,0
2,4,0.278481,19611,19,0,1,Q,1,0
3,3,0.150289,13583,22,0,1,Q,1,0
4,1,0.016573,18042,2,0,1,Q,1,0
...,...,...,...,...,...,...,...,...,...
32701,2,0.028365,23619,2,0,1,M,1,0
32702,1,0.000606,3935,1,0,1,M,1,0
32703,2,0.040452,30163,4,0,1,M,1,0
32704,2,0.000000,29012,4,0,1,M,1,0


In [33]:
dat_new.dtypes

gravedad                  int64
pct_mortalidad_norma    float64
edad_dias                 int32
numproc                    int8
potencial_ambul           int64
proc                      int64
tipgrd                   object
tiping                    int64
exitus                    int64
dtype: object

# 3 oversampling / subsampling

In [31]:
dat_numeric = dat_new.drop(categorical_variables, axis=1)
dat_numeric

,pct_mortalidad_norma,edad_dias,numproc,proc,exitus
0,0.408730,12596.0,21.0,1,0
1,0.178418,20973.0,22.0,1,0
2,0.278481,19611.0,19.0,1,0
3,0.150289,13583.0,22.0,1,0
4,0.016573,18042.0,2.0,1,0
...,...,...,...,...,...
32701,0.028365,23619.0,2.0,1,0
32702,0.000606,3935.0,1.0,1,0
32703,0.040452,30163.0,4.0,1,0
32704,0.000000,29012.0,4.0,1,0
